# CAKE - Genetic Algorithm POC 

Simple Test case we have tested for memeory constraints. 
<pre>
machine = {"m1" : {"ram" : 4, "HDD" : 500, "cores" : 3 },
     "m2" : {"ram" : 12, "HDD" : 1000, "cores" : 5 } 
    }
jobs = {"j1" : {"ram" : 200, "HDD" : 100, "cores" : 3},
     "j2" : {"ram" : 500, "HDD" : 50, "cores" : 5}, 
     "j3" : {"ram" : 100, "HDD" : 30, "cores" : 3}, 
     "j4" : {"ram" : 1024, "HDD" : 1, "cores" : 5}
    }

schedule_1 = {"m1" : ["j1", "j3"],
            "m2" : ["j2", "j4"]
           }
schedule_2 = {"m1" : ["j1", "j2"],
            "m2
</pre>

In [ ]:
import random

from deap import base
from deap import creator
from deap import tools
import matplotlib.pyplot as plt
%matplotlib inline 

In [131]:
m = {"m1" : {"ram" : 4, "HDD" : 500, "cores" : 3 },
     "m2" : {"ram" : 12, "HDD" : 1000, "cores" : 5 } 
    }
j = {"j1" : {"ram" : 200, "HDD" : 100, "cores" : 3},
     "j2" : {"ram" : 500, "HDD" : 50, "cores" : 5}, 
     "j3" : {"ram" : 100, "HDD" : 30, "cores" : 3}, 
     "j4" : {"ram" : 1024, "HDD" : 1, "cores" : 5}
    }

schedule_1 = {"m1" : ["j1", "j3"],
            "m2" : ["j2", "j4"]
           }
schedule_2 = {"m1" : ["j1", "j2"],
            "m2" : ["j3", "j4"]
           }

In [132]:
def evaluate_schedule(schedule, m , j):
    l = len(schedule)
    fit = 0
    for key, values in schedule.items():
        max_core = 0
        t_r = 0
        t_h = 0
        m_c = m[key]
        for item in values:
            j_c = j[item]
            if j_c["cores"] > max_core:
                max_core = j_c["cores"]
            t_r += j_c["ram"]
            t_h += j_c["HDD"]
        if (max_core <= m_c["cores"]) and (t_r <= m_c["ram"] * 1024) and (t_h <= m_c["HDD"]):
            fit += 1
    score = (fit/l)*100
    #print("Score : %f"%score)
    
    #check if this is a complete schedule
    allJobs = list(j.keys())
    allMachines = list(m.keys())
    scheduled_jobs = list()
    for machine in allMachines:
        jobs_on_machine = schedule[machine]
        for job in jobs_on_machine:
            if job not in scheduled_jobs:
                scheduled_jobs.append(job)
    """
    for job in allJobs:
        valid = False
        for machine in allMachines:
            if job in m[machine]:
                valid = True
                break;
            else:
                continue
        if valid == False:
            break
    """
    valid = False
    if len(allJobs) == len(scheduled_jobs):
        valid = True
    return (score, valid)
    #return score

In [133]:
import math as ma
def crossover_1(schedule_1,schedule_2):
    """
    crosses over jobs on same machine in different schedule. example below :
    old schedule_1 :  {'m1': ['j1', 'j2'], 'm2': ['j3', 'j4']}
    old schedule_2 :  {'m1': ['j4', 'j3'], 'm2': ['j2', 'j1']}

    new schedule_1 :  {'m1': ['j1', 'j3'], 'm2': ['j3', 'j1']}
    new schedule_2 :  {'m1': ['j4', 'j2'], 'm2': ['j2', 'j4']}
    """
    l = len(schedule_1)
    n_s_1 = {}
    n_s_2 = {}
    for i in range(l):
        ith_machine_jobs_1 = schedule_1["m"+str(i+1)]
        ith_machine_jobs_2 = schedule_2["m"+str(i+1)]
        slice_point = 0
        if len(ith_machine_jobs_1) > len(ith_machine_jobs_2):
            slice_point = ma.ceil(len(ith_machine_jobs_1)/2)
        else:
            slice_point = ma.ceil(len(ith_machine_jobs_2)/2)
            
        new_1 = [item for item in ith_machine_jobs_1[:slice_point]]
        for item in ith_machine_jobs_2[slice_point:]:
            new_1.append(item)
        n_s_1.update({"m"+str(i+1) : new_1 })
        
        new_2 = [item for item in ith_machine_jobs_2[:slice_point]]
        for item in ith_machine_jobs_1[slice_point:]:
            new_2.append(item)
        
        n_s_2.update({"m"+str(i+1) : new_2 })
    """
    print("old schedule_1 : ",schedule_1)
    print("old schedule_2 : ",schedule_2)
    print()
    print("new schedule_1 : ",n_s_1)
    print("new schedule_2 : ",n_s_2)
    """
    return n_s_1, n_s_2

In [134]:
def crossover_2(s_1, s_2):
    """
    crosses over jobs of the machine in the second part of this schedule with the second part of other schedule.
    example below :
    old schedule_1 :  {'m1': ['j1', 'j2'], 'm2': ['j3', 'j4']}
    old schedule_2 :  {'m1': ['j4', 'j3'], 'm2': ['j2', 'j1']}

    new schedule_1 :  {'m1': ['j1', 'j2'], 'm2': ['j2', 'j1']} 
    new schedule_2 :  {'m1': ['j4', 'j3'], 'm2': ['j3', 'j4']}
    """
    l = len(s_1)
    n_s_1 = {}
    n_s_2 = {}
    slice_point = ma.ceil(len(s_1)/2)
    for i in range(l):
        if i < slice_point:
            n_s_1.update({"m"+str(i+1) : s_1["m"+str(i+1)]})
            n_s_2.update({"m"+str(i+1) : s_2["m"+str(i+1)]})
        else:
            n_s_1.update({"m"+str(i+1) : s_2["m"+str(i+1)]})
            n_s_2.update({"m"+str(i+1) : s_1["m"+str(i+1)]})
    """
    print("old schedule_1 : ",s_1)
    print("old schedule_2 : ",s_2)
    print()
    print("new schedule_1 : ",n_s_1)
    print("new schedule_2 : ",n_s_2)
    """
    return n_s_1, n_s_2

In [135]:
import random
def crossover(s_1, s_2):
    choice = random.randint(0,1)
    #print(choice)
    if choice == 0:
        return crossover_1(s_1, s_2)
    else:
        return crossover_2(s_1, s_2)

In [136]:
#generate the initial population
def generateInitalPopulation_randomly(machine_details, job_details, numberOfSchedules):
    randomSchedules = list()
    available_machines = list(machine_details.keys())
    submitted_jobs = list(job_details.keys())
    
    jobs_per_machine = ma.ceil(len(submitted_jobs) / len(available_machines))
    
    #print(submitted_jobs)
    #print(available_machines)
    
    for i in range(numberOfSchedules):
        submitted_jobs = list(job_details.keys())
        schedule = {}
        for machine in available_machines:
            schedule.update({ machine : list()})
            for j in range(jobs_per_machine):
                if(len(submitted_jobs) <= 0):
                    break
                index = random.randint(0,len(submitted_jobs)-1)
                job = submitted_jobs.pop(index)#removing scheduled item
                schedule[machine].append(job)
        randomSchedules.append(schedule)
    return randomSchedules

In [137]:
#generates next population using crossover
def generateNextPopulationUsingCrossOver(df, m, j, n):
    schedule = df['schedule']
    fitness_score = df['fitness score']
    new_Schedules = list()
    while(len(new_Schedules) < n):
        i1 = random.randint(0,n-1)
        i2 = random.randint(0,n-1)
        crossed_over_schedules = crossover(schedule[i1], schedule[i2])
        for item in crossed_over_schedules:
            if not len(new_Schedules) >= n:
                new_Schedules.append(item)
    return new_Schedules

In [199]:
#print(random.random())
def mutate(schedules, j, prob):
    print(schedules)
    allJobs = list(j.keys())
    for i in range(len(schedules)):
        #print(i)
        for key in schedules[i].keys():#length of each schedule i.e, number of machines
            #print(schedules[i])
            for k in range(len(schedules[i][key])):#number of jobs on each machine
                #print("K : ",k)
                if random.random() < prob:
                    possible_values = list(allJobs)
                    possible_values.remove(schedules[i][key][k])
                    index = random.randint(0,len(possible_values) - 1)
                    schedules[i][key][k] = possible_values[index]
    return schedules
#mutate([schedule_1,schedule_2], j, 0.5)

In [202]:
import pandas as pd
import sys
def demoGA():
    #random.seed(64)
    CXPB, MUTPB, NGEN, POP_SIZE = 0.5, 0.2, 500, 2
    initialRandomSchedules = generateInitalPopulation_randomly(m, j, POP_SIZE)
    #print(initialRandomSchedules)
    fitnesses = list()
    for schedule in initialRandomSchedules:
        fit_score = evaluate_schedule(schedule, m , j)
        #fitnesses.append(fit_score)
        if fit_score[0] == 100 and (fit_score[1] == True):
                print("Solution found : ", schedule)
                return
        else:
            fitnesses.append(fit_score)
 
    #print(fitnesses)
    #for schedule, fitness in zip(intialRandomScehules, fitnesses):
    print("Initial Schedules : ")
    df = pd.DataFrame(data={'schedule' : initialRandomSchedules, 'fitness score' : fitnesses})
    df = df[['schedule', 'fitness score']]
    df = df.sort_values(axis=0,by='fitness score',ascending=False)
    print(df)
    
    for i in range(NGEN):
        print("\nGeneration : ",i)
        new_Schedule = generateNextPopulationUsingCrossOver(df, m, j, POP_SIZE)
        new_Schedule = mutate(new_Schedule, j, MUTPB)
        fitnesses = list()
        for schedule in new_Schedule:
            fit_score = evaluate_schedule(schedule, m , j)
            #print(fit_score)
            if fit_score[0] == 100 and (fit_score[1] == True):
                print("Solution found : ", schedule)
                return
            else:
                fitnesses.append(fit_score)
        
        df = pd.DataFrame(data={'schedule' : new_Schedule, 'fitness score' : fitnesses})
        df = df[['schedule', 'fitness score']]
        df = df.sort_values(axis=0,by='fitness score',ascending=False)  
        print(df)
    
if __name__ == "__main__":
    demoGA()

Initial Schedules : 
                                   schedule fitness score
0  {'m1': ['j3', 'j2'], 'm2': ['j1', 'j4']}  (50.0, True)
1  {'m1': ['j3', 'j2'], 'm2': ['j1', 'j4']}  (50.0, True)

Generation :  0
[{'m1': ['j3', 'j2'], 'm2': ['j1', 'j4']}, {'m1': ['j3', 'j2'], 'm2': ['j1', 'j4']}]
                                   schedule fitness score
0  {'m1': ['j3', 'j2'], 'm2': ['j1', 'j4']}  (50.0, True)
1  {'m1': ['j3', 'j2'], 'm2': ['j1', 'j4']}  (50.0, True)

Generation :  1
[{'m1': ['j3', 'j2'], 'm2': ['j1', 'j4']}, {'m1': ['j3', 'j2'], 'm2': ['j1', 'j4']}]
                                   schedule  fitness score
0  {'m1': ['j4', 'j4'], 'm2': ['j1', 'j1']}  (50.0, False)
1  {'m1': ['j4', 'j4'], 'm2': ['j1', 'j1']}  (50.0, False)

Generation :  2
[{'m1': ['j4', 'j4'], 'm2': ['j1', 'j1']}, {'m1': ['j4', 'j4'], 'm2': ['j1', 'j1']}]
                                   schedule  fitness score
0  {'m1': ['j4', 'j2'], 'm2': ['j1', 'j1']}  (50.0, False)
1  {'m1': ['j4', 'j4'], 'm2': 